In [1]:
import torch
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import syft as sy
import numpy as np
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
james = sy.VirtualWorker(hook, id="james")
import sys

In [2]:
8/5

1.6

In [25]:
lx = 8
dec_ld = 1
ld = 3*dec_ld

In [28]:
i = 0
m = 1000
for j in range(m):
    #print('--')
    x = torch.tensor([1. * 2**lx]).fix_prec(precision_fractional=dec_ld).share(alice, bob, crypto_provider =james)
    y = torch.tensor([1. * 2**lx]).fix_prec(precision_fractional=dec_ld).share(alice, bob, crypto_provider =james)
    z = (x * y)
    #print(x.child.child.child.virtual_get())
    #print(y.child.child.child.virtual_get())
    #print(z.child.child.child.virtual_get())
    _z = z.child.child.child.virtual_get()
    r = z.get().float_prec()
    if (-1 < (r - torch.tensor([1. * 2**(2*lx)])) < 1).all():
        i += 1
    #else:
    #    print(r)

print(i)
    

999


In [27]:
1-2**(2*(lx+ld) - 32)

0.9990234375

In [35]:
1-2**(2*(16+10) - 64)

0.999755859375

In [29]:
2**16

65536

In [ ]:
for i in range(0):
    print('--')
    x = torch.tensor([1., 2, -3, 4, 5233, 6]).fix_prec().share(alice, bob, crypto_provider =james)
    z = x  > 0
    #print(x.child.child.child.virtual_get())
    #print(y.child.child.child.virtual_get())
    print(z.child.child.child.virtual_get())
    r = z.get().float_prec()
    print(r)

In [3]:
x = torch.tensor([1444506136, 1520521833, 1808833055,   75941325,  878293374, 1074573176])
y = torch.tensor([ 703977511,  630961814,  347650592, 2087542322, 1294190273, 1108910471])
z = x + y
truncate = 1000
x_trunc = x / truncate
y_trunc = y / truncate
z_trunc = x_trunc + y_trunc
field = (2 ** 31) - 1

In [17]:
(
    int(75941325/truncate) 
    + int((2087542322 - field)/truncate)  
) % (field/truncate)

16000.0

In [30]:
(1074573176 + 1108910471) % field, int(field/truncate)

(36000000, 2147483)

In [31]:
(
    int((1074573176 - field)/truncate) % int(field/truncate)
    + int((1108910471 - field)/truncate)  % int(field/truncate)
) % int(field/truncate)

36000

In [26]:
(
    int((1074573176 - field)/truncate) 
    + int((1108910471 - field)/truncate)  
) #% int(field)

-2111483

In [20]:
int((1108910471 - field)/truncate) % field

2146445074

In [16]:
z % field

tensor([ 1000000,  4000000,  9000000, 16000000, 25000000, 36000000])

In [15]:
z_trunc % field

tensor([2148483, 2151482, 2156483, 2163483, 2172483, 2183483])

In [4]:
alice._objects

{26779938685: tensor([ 914486443,  846495944,   88850875,   29141876, 1888707648,  340937636]),
 39357936351: tensor([ 758063575,  371863848,  171881930, 2046968862, 1067871371, 1457163631])}

In [14]:
bob._objects

{16952206870: tensor([1713286492, 1617072994])}

In [11]:
(1971299777 + 176184870) % 2147483647

1000

In [18]:
L=21
crypto_provider=james

In [20]:
u = torch.zeros(1).long().share(alice, bob, field=L, crypto_provider=crypto_provider).child
u

[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:31188863175 -> alice:33721718912]
	-> (Wrapper)>[PointerTensor | me:57736495574 -> bob:98808237073]
	*crypto provider: james*

In [24]:
u = (
        torch.zeros(1)
        .long()
        .send(alice)
        .share(alice, bob, field=L, crypto_provider=crypto_provider)
        .get()
        .child
    )
u.get()

tensor([0])

In [44]:
Q_BITS = 8
def decompose(tensor):
    """decompose a tensor into its binary representation."""
    n_bits = Q_BITS
    powers = torch.arange(n_bits)
    if hasattr(tensor, "child") and isinstance(tensor.child, dict):
        powers = powers.send(*list(tensor.child.keys())).child
    for i in range(len(tensor.shape)):
        powers = powers.unsqueeze(0)
    tensor = tensor.unsqueeze(-1)
    moduli = 2 ** powers
    tensor = torch.fmod((tensor / moduli.type_as(tensor)), 2)
    #tensor = torch.fmod(((tensor + 2 ** n_bits) / moduli.type_as(tensor)), 2)
    return tensor

In [48]:
decompose(torch.tensor(148))

tensor([0, 0, 1, 0, 1, 0, 0, 1])